In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

2023-07-08 14:58:53.851926: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(-1, 784).astype("float32") / 255.0
x_test = x_test.reshape(-1, 784).astype("float32") / 255.0
y_train = to_categorical(y_train, num_classes=10)
y_test = to_categorical(y_test, num_classes=10)

# Define the SNN layer

In [3]:
class SpikingLayer(tf.keras.layers.Layer):
    def __init__(self, units, **kwargs):
        super(SpikingLayer, self).__init__(**kwargs)
        self.units = units

    def build(self, input_shape):
        self.kernel = self.add_weight("kernel", shape=[input_shape[-1], self.units])

    def call(self, inputs):
        spikes = tf.cast(inputs > 0, dtype=tf.float32)
        output = tf.matmul(spikes, self.kernel)
        return output

In [4]:
model = tf.keras.Sequential([
    layers.Dense(units=32),
    SpikingLayer(units=128),  # 第一层SNN层
    layers.Activation('relu'),
    #SpikingLayer(units=128),  # 第二层SNN层
    #layers.Activation('relu'),
    layers.Dense(units=10),  # 输出层
    layers.Activation('softmax')
])

2023-07-08 14:58:55.799428: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
model.compile(
    optimizer="adam", 
    loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True), 
    metrics=["accuracy"]
)

model.fit(
    x_train, 
    y_train, 
    batch_size=5096, 
    epochs=300, 
    validation_data=(x_test, y_test),
    verbose = 0
)

test_loss, test_acc = model.evaluate(x_test, y_test)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_acc)

/opt/conda/lib/python3.10/site-packages/keras/backend.py:5534: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


313/313 [==============================] - 0s 816us/step - loss: 0.7974 - accuracy: 0.7323
Test Loss: 0.7974334359169006
Test Accuracy: 0.7322999835014343


In [6]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                25120     
                                                                 
 spiking_layer (SpikingLayer  (None, 128)              4096      
 )                                                               
                                                                 
 activation (Activation)     (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 10)                1290      
                                                                 
 activation_1 (Activation)   (None, 10)                0         
                                                                 
Total params: 30,506
Trainable params: 30,506
Non-trainable params: 0
____________________________________________________

In [7]:
model.save("SNN_MNIST.h5")

# hls4ml Config Part

In [8]:
from tensorflow.keras.models import load_model

model = load_model("SNN_MNIST.h5", custom_objects={'SpikingLayer': SpikingLayer})

In [9]:
test_loss, test_acc = model.evaluate(x_test, y_test)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_acc)
model.summary()

313/313 [==============================] - 0s 806us/step - loss: 0.7974 - accuracy: 0.7323
Test Loss: 0.7974334359169006
Test Accuracy: 0.7322999835014343
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                25120     
                                                                 
 spiking_layer (SpikingLayer  (None, 128)              4096      
 )                                                               
                                                                 
 activation (Activation)     (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 10)                1290      
                                                                 
 activation_1 (Activation)   (None, 10)                0         
                                 

In [10]:
import hls4ml

class HSpikingLayer(hls4ml.model.layers.Layer):
    def initialize(self):
        inp = self.get_input_variable()
        shape = inp.shape
        dims = inp.dim_names
        self.add_output_variable(shape, dims)

    def build(self):
        input_var = self.get_input_variable()
        output_var = self.get_output_variable()
        output_var.add_assignment(f'spikes = nnet::greater<{input_var.precision}, 0>({input_var()});')
        output_var.add_assignment(f'{output_var()} = nnet::matmul<{input_var.precision}, {output_var.precision}, {output_var.target_name}>({output_var()}, spikes, {self.kernel()});')

In [11]:
def parse_spiking_layer(keras_layer, input_names, input_shapes, data_reader):
    layer = {}
    layer['class_name'] = 'HSpikingLayer'
    layer['name'] = keras_layer['config']['name']  
    layer['units'] = keras_layer['config']['units'] 

    if input_names is not None:
        layer['inputs'] = input_names

    return layer, input_shapes

In [12]:
def register_custom_layer():
    hls4ml.converters.register_keras_layer_handler('SpikingLayer', parse_spiking_layer)
    hls4ml.model.layers.register_layer('HSpikingLayer', HSpikingLayer)

In [13]:
register_custom_layer()

In [15]:
import hls4ml
import plotting

config = hls4ml.utils.config_from_keras_model(model, granularity='name')
#config['InputShape'] = {'spiking_layer_input': (None, 784)}
config['Model']['Precision'] = 'ap_fixed<12,6>'
config['Model']['ReuseFactor'] = 1
'''
for Layer in config['LayerName'].keys():
    config['LayerName'][Layer]['Strategy'] = 'Latency'
    config['LayerName'][Layer]['ReuseFactor'] = 1
    #config['LayerName'][Layer]['Precision'] = 'ap_fixed<8,4>'
'''
config['LayerName']['activation_1']['Strategy'] = 'Stable'
'''
for layer in ['conv1', 'conv2'] :
    config['LayerName'][layer]['Precision'] = 'ap_fixed<8,4>'
'''
print("-----------------------------------")
plotting.print_dict(config)
print("-----------------------------------")

cfg = hls4ml.converters.create_config(backend='VivadoAccelerator')
cfg['IOType'] = 'io_stream'
cfg['HLSConfig'] = config
cfg['KerasModel'] = model
cfg['OutputDir'] = 'AlexNet_Alevo50'
cfg['Board'] = 'alveo-u50'

hls_model = hls4ml.converters.keras_to_hls(cfg)

hls_model.compile()

Interpreting Sequential
Topology:
Layer name: dense_input, layer type: InputLayer, input shapes: [[None, 784]], output shape: [None, 784]
Layer name: dense, layer type: Dense, input shapes: [[None, 784]], output shape: [None, 32]
Layer name: spiking_layer, layer type: HSpikingLayer, input shapes: [[None, 32]], output shape: [[None, 32]]
Layer name: activation, layer type: Activation, input shapes: [[[None, 32]]], output shape: [[None, 32]]
Layer name: dense_1, layer type: Dense, input shapes: [[[None, 32]]], output shape: [10]
Layer name: activation_1, layer type: Softmax, input shapes: [[10]], output shape: [10]
-----------------------------------
Model
  Precision:         ap_fixed<12,6>
  ReuseFactor:       1
  Strategy:          Latency
  BramFactor:        1000000000
  TraceOutput:       False
LayerName
  dense_input
    Trace:           False
    Precision
      result:        fixed<16,6>
  dense
    Trace:           False
    Precision
      result:        fixed<16,6>
      weig

In [17]:
import os

os.environ['PATH'] = os.environ['XILINX_VIVADO'] + '/bin:' + os.environ['PATH']
os.environ['LD_PRELOAD'] = '/lib/x86_64-linux-gnu/libudev.so.1'

In [18]:
hls_model.build(csim=False, export=True, bitfile=True)


****** Vivado(TM) HLS - High-Level Synthesis from C, C++ and SystemC v2019.2 (64-bit)
  **** SW Build 2708876 on Wed Nov  6 21:39:14 MST 2019
  **** IP Build 2700528 on Thu Nov  7 00:09:20 MST 2019
    ** Copyright 1986-2019 Xilinx, Inc. All Rights Reserved.

source /opt/Xilinx/Vivado/2019.2/scripts/vivado_hls/hls.tcl -notrace
INFO: Applying HLS Y2K22 patch v1.2 for IP revision
INFO: [HLS 200-10] Running '/opt/Xilinx/Vivado/2019.2/bin/unwrapped/lnx64.o/vivado_hls'
INFO: [HLS 200-10] For user 'jovyan' on host '6307b0c947c6' (Linux_x86_64 version 4.15.0-212-generic) on Sat Jul 08 15:04:23 UTC 2023
INFO: [HLS 200-10] In directory '/home/jovyan/Internship_Waseda/hls4ml/SNN/AlexNet_Alevo50'
Sourcing Tcl script 'build_prj.tcl'
INFO: [HLS 200-10] Opening project '/home/jovyan/Internship_Waseda/hls4ml/SNN/AlexNet_Alevo50/myproject_prj'.
INFO: [HLS 200-10] Adding design file 'firmware/myproject_axi.cpp' to the project
INFO: [HLS 200-10] Adding design file 'firmware/myproject.cpp' to the projec

ERROR: [BD 5-390] IP definition not found for VLNV: xilinx.com:hls:myproject_axi:1.0 


CSynthesis report not found.
Vivado synthesis report not found.
Cosim report not found.
Timing report not found.


{}